In [6]:
import os
import urllib.request as request
from data_science import logger
import zipfile
from box import ConfigBox

In [2]:
import os
import sys

In [3]:
%pwd

'/Users/saiaakash/Documents/mlops/DS_main/research'

In [4]:
project_root = "/Users/saiaakash/Documents/mlops/DS_main"

# Add the src directory to the Python path
src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    sys.path.insert(0, src_path)

# Print to verify
print(f"Project root: {project_root}")
print(f"Python path: {sys.path}")

Project root: /Users/saiaakash/Documents/mlops/DS_main
Python path: ['/opt/anaconda3/envs/mlpipe/lib/python310.zip', '/opt/anaconda3/envs/mlpipe/lib/python3.10', '/opt/anaconda3/envs/mlpipe/lib/python3.10/lib-dynload', '', '/opt/anaconda3/envs/mlpipe/lib/python3.10/site-packages', '/Users/saiaakash/Documents/mlops/DS_main/src']


In [5]:
import pandas as pd

In [8]:
data=pd.read_csv("/Users/saiaakash/Documents/mlops/DS_main/artifacts/data_ingestion/winequality-red.csv")
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [10]:
data.shape

(1599, 12)

In [11]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    unzip_dir: Path
    status_file: Path
    all_schema: dict


In [12]:
from data_science.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH
from data_science.utils.common import read_yaml, create_directories

In [13]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH, schema_filepath=SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config=self.config.data_validation
        create_directories([config.root_dir])
        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            unzip_dir=config.unzip_dir,
            status_file=config.status_file,
            all_schema=self.schema  
        )
        return data_validation_config


In [14]:
import os
import urllib.request as request
from data_science import logger
import zipfile
from box import ConfigBox


In [15]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_files(self):
        try:
            validation_status = None
            data=pd.read_csv(self.config.unzip_dir)
            all_cols=list(data.columns)

            all_schema=self.config.all_schema.keys()

            for col in all_cols:
                if col not in all_schema:
                    validation_status=False
                    with open(self.config.status_file,'w') as f:
                        f.write(f"Validation Status {validation_status}")
                else:
                    validation_status=True
                    with open(self.config.status_file,'w') as f:
                        f.write(f"Validation Status {validation_status}")
            return validation_status
        except Exception as e:
            raise e


In [17]:
try:
    config=ConfigurationManager()
    data_validation_config=config.get_data_validation_config()
    data_validation=DataValidation(data_validation_config)
    data_validation.validate_all_files()
except Exception as e:
    raise e

[2025-07-28 11:36:47,520: ERROR: common]: Error loading yaml file: config/config.yaml with error: [Errno 2] No such file or directory: 'config/config.yaml'


FileNotFoundError: [Errno 2] No such file or directory: 'config/config.yaml'